# Homework 04
Austin Derrow-Pinion

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
train = np.load('./Data/MNIST_train_100.npz')
test = np.load('./Data/MNIST_test_100.npz')
print("Train keys: ", train.keys())
print("Test keys:  ", test.keys())

Train keys:  ['train_labels', 'train_images']
Test keys:   ['test_images', 'test_labels']


In [70]:
train_images = train['train_images']
test_images = test['test_images']

train_labels = train['train_labels']
test_labels = test['test_labels']

print("Train images shape: ", train_images.shape)
print("Train labels shape: ", train_labels.shape)

print("\nTest images shape: ", test_images.shape)
print("Test labels shape: ", test_labels.shape)

Train images shape:  (100, 28, 28)
Train labels shape:  (100,)

Test images shape:  (100, 28, 28)
Test labels shape:  (100,)


In [36]:
print("train_labels:\n", train_labels)
print("test_labesl:\n", test_labels)

train_labels:
 [4 9 9 9 9 9 4 9 9 9 4 4 4 4 4 9 9 4 9 4 9 4 4 4 4 9 9 9 9 9 9 4 4 4 9 9 4
 9 9 4 9 4 9 4 9 4 4 9 9 9 9 9 4 9 4 4 4 9 9 9 9 9 9 9 9 9 9 9 4 9 9 9 9 4
 9 9 4 9 4 4 9 9 4 9 4 4 4 4 4 4 9 9 4 4 4 9 9 9 4 9]
test_labesl:
 [9 9 9 9 4 4 4 4 9 9 4 4 4 4 9 4 9 4 9 9 9 9 4 9 4 9 9 4 4 4 9 4 4 9 4 4 9
 4 4 9 9 9 4 4 9 4 4 9 9 9 4 4 4 4 9 4 9 9 4 9 4 9 4 9 4 9 9 4 9 4 4 9 9 9
 4 4 4 9 4 4 9 9 4 4 4 9 4 4 9 9 9 9 4 4 9 9 4 4 9 4]


#### Part (a): Using TensorFlow train a logistic regression classifier for predicting the number corresponding to the image of a handwritten character.

In [102]:
# prepare input data
x_test = test_images.reshape(100, 28*28).astype(np.float32)

x_train = train_images.reshape(100, 28*28)
print("x_train shape: ", x_train.shape)
x_train[0][235:240]

x_train shape:  (100, 784)


array([ 0.91372555,  0.98823535,  0.99215692,  0.43529415,  0.        ], dtype=float32)

In [83]:
# prepare output data
y_test = pd.get_dummies(test_labels).values.astype(np.float32)

y_train = pd.get_dummies(train_labels).values.astype(np.float32)
print("y_train shape: ", y_train.shape)
y_train[0:3]

y_train shape:  (100, 2)


array([[ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)

In [105]:
# Tensorflow computation graph

# y = softmax(x * w + b)
x = tf.constant(x_train, dtype='float32', shape=[100, 784])
y = tf.constant(y_train, dtype='float32', shape=[100, 2])

w = tf.Variable(tf.truncated_normal([784, 2], stddev=0.001))
b = tf.Variable(tf.truncated_normal([1, 2], stddev=0.001))

# output of function
y_pred = tf.nn.softmax(tf.matmul(x, w) + b)

# test predictions
x_test_tf = tf.constant(x_test, dtype='float32', shape=[100, 784])
y_pred_test = tf.nn.softmax(tf.matmul(x_test_tf, w) + b)

# error loss function
MSE = tf.reduce_mean(tf.square(y_pred - y))

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.01, epsilon=1e-10).minimize(MSE)

init = tf.initialize_all_variables()

In [106]:
sess = tf.Session()
sess.run(init)

In [107]:
MAXSTEPS = 20000
for step in range(MAXSTEPS + 1):
    (_, mse, w0, b0) = sess.run([optimizer, MSE, w, b])
    if (step % 1000) == 0:
        print('step = %-5d MSE = %-10f' % (step, mse))
print('done!')

step = 0     MSE = 0.250284  
step = 1000  MSE = 0.000052  
step = 2000  MSE = 0.000015  
step = 3000  MSE = 0.000007  
step = 4000  MSE = 0.000004  
step = 5000  MSE = 0.000002  
step = 6000  MSE = 0.000001  
step = 7000  MSE = 0.000001  
step = 8000  MSE = 0.000000  
step = 9000  MSE = 0.000000  
step = 10000 MSE = 0.000000  
step = 11000 MSE = 0.000000  
step = 12000 MSE = 0.000000  
step = 13000 MSE = 0.000000  
step = 14000 MSE = 0.000000  
step = 15000 MSE = 0.000000  
step = 16000 MSE = 0.000000  
step = 17000 MSE = 0.000000  
step = 18000 MSE = 0.000000  
step = 19000 MSE = 0.000000  
step = 20000 MSE = 0.000000  
done!


In [108]:
# make predictions
predictions = sess.run(y_pred)
predictions[0:3]

array([[  9.99963880e-01,   3.60827325e-05],
       [  3.51603594e-05,   9.99964833e-01],
       [  1.01299620e-05,   9.99989867e-01]], dtype=float32)

In [109]:
# check mean square error
mse = ((y_train - predictions) ** 2).mean()
rmse = np.sqrt(mse)
print('MSE = ', mse)
print('RMSE = ', rmse)
print('output\'s standard deviation = ', y_train.std())
print('difference between RMSE of model and std of LogRatios: ', (y_train.std() - rmse))

MSE =  9.35573e-10
RMSE =  3.05871e-05
output's standard deviation =  0.5
difference between RMSE of model and std of LogRatios:  0.499969


#### Part (b): Repear part (a) using Sklearn.

In [110]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
LgR = LogisticRegression(solver='lbfgs', multi_class='multinomial')

In [111]:
LgR.fit(x_train, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [112]:
# predict species probabilities
predictions_sk = LgR.predict_proba(x_train)
predictions_sk[0:3]

array([[ 0.82771958,  0.17228042],
       [ 0.13397693,  0.86602307],
       [ 0.10244145,  0.89755855]])

In [113]:
mse_sk = mean_squared_error(y_train, predictions_sk)
print("Sklearn mse: ", mse_sk)
print("Sklearn rmse: ", np.sqrt(mse_sk))

Sklearn mse:  0.0192015802169
Sklearn rmse:  0.138569766605


#### Part (c): Compare Tensorflow's MSE to that of Sklearn.

In [114]:
print("TensorFlow's MSE: ", mse)
print("Sklearn's MSE: ", mse_sk)

TensorFlow's MSE:  9.35573e-10
Sklearn's MSE:  0.0192015802169


#### Part (d): Compare Tensorflow's accuracy rate to that of Sklearn.

accuracy = (# of correct predictions) / (total # of predictions)

error rate = 1 - accuracy

In [115]:
accuracy = accuracy_score(np.argmax(y_train, axis=1), np.argmax(predictions, axis=1))
accuracy_sk = accuracy_score(np.argmax(y_train, axis=1), np.argmax(predictions_sk, axis=1))

print("TensorFlow's accuracy: ", accuracy)
print("Sklearn's accuracy: ", accuracy_sk)

TensorFlow's accuracy:  1.0
Sklearn's accuracy:  1.0


#### Part (e): Repeat parts (c) and (d) using the test data provided. (Do not retrain your classifier on the test data!)

In [116]:
predictions_test = sess.run(y_pred_test)
predictions_test_sk = LgR.predict_proba(x_test)

# check mean square error
mse_test = mean_squared_error(y_test, predictions_test)
mse_test_sk = mean_squared_error(y_test, predictions_test_sk)

print("TensorFlow's MSE: ", mse_test)
print("Sklearn's MSE: ", mse_test_sk)

TensorFlow's MSE:  0.100542
Sklearn's MSE:  0.0800912534225


In [117]:
accuracy_test = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions_test, axis=1))
accuracy_test_sk = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions_test_sk, axis=1))

print("TensorFlow's accuracy: ", accuracy_test)
print("Sklearn's accuracy: ", accuracy_test_sk)

TensorFlow's accuracy:  0.87
Sklearn's accuracy:  0.89
